# TimeEmbedding class
* ./common/time_layers.py

embedding층 역할: 단어 입력 -> 면 번째 등장?: 단어 id <br>
embedding 층에 Weight matrix 가 있으면 <br>
Weight Matrix에서 단어에 해당하는 행 뽑아냄. <br>
그 뽑아낸 행이 단어의 벡터표현. 그걸 RNN층으로 보내라 <br>

In [ ]:
class TimeEmbedding:
    def __init__(self, W): # embedding층 입력: weight
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.layers = None
        self.W = W

    def forward(self, xs): # 순전파, 입력: 밑에서 올라오는 단어들
        N, T = xs.shape # batch_size, time_block의 길이
        V, D = self.W.shape # (weight mat안에는 단어의 헥터표현이 각 행에 저장됨); 행개수 = 단어개수, weight mat의 열 크기 = 벡터의 크기(단어의벡터표현)

        out = np.empty((N, T, D), dtype='f') # 3차원 tensor # 단어 id에 해당하는 ....를 채워 넣는...
        # 전체적인 shape은 batch_size * Time_size
        self.layers = [] # 공리스트에 임베딩 인스턴스 차례대로 채워넣을 것

        for t in range(T): # timeblock 길이 (timesize)
            layer = Embedding(self.W) # 임베딩인스턴스를 레이어로 두고
            out[:, t, :] = layer.forward(xs[:, t]) # 순전파, 열자리에 layer넣고 순전파
            self.layers.append(layer) # 방금 만ㄴ든 인스턴스 추가

        return out

    def backward(self, dout): # 역전파, 위에서 흘러들어온 미분 필요
        N, T, D = dout.shape # out의 shape과 동일

        grad = 0
        for t in range(T):
            layer = self.layers[t]
            layer.backward(dout[:, t, :])
            grad += layer.grads[0]

        self.grads[0][...] = grad
        return None # 밑으로 흘러보낼 필요가 없기 때문에 None

# TimeAffine class
* ./common/time_layers.py

In [ ]:
class TimeAffine:
    def __init__(self, W, b):
        self.params = [W, b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None

    def forward(self, x): # 순전파 하려면 밑에서 흘러들어온 데이터 필요 (밑: time rnn 층, 거기서 나오는게 hidden_state, 이건 위로도 보내고, 동시에 순환을 시킴)
        N, T, D = x.shape
        W, b = self.params

        rx = x.reshape(N*T, -1) # 각각에 대한 affine변환을 하지 않고 한 번에 하기 위한 행렬 변환
        out = np.dot(rx, W) + b # 한 번에 오른쪽에 Weight mat 곱하고 편향벡터 더하기 (out: affine 변환 결과)
            # affine층에서 나가는건 vocab 사이즈
        self.x = x
        return out.reshape(N, T, -1) # Batch_size * Time_size * vocab_size, 이제 이걸 위 층으로 보냄.

    def backward(self, dout): # 역전파, 위에서 흘러들어온 미분 필요. (위: time Softmax 층)
        x = self.x
        N, T, D = x.shape # shape은 위(time Softmax층)로 보낼 때랑 동일.
        W, b = self.params

        dout = dout.reshape(N*T, -1) # reshape해서 3차원 텐서를 행렬로 만듦. (세로: Batch_size * Time_size, 가로: vocab_size)
            # softmax층에서 내려온 거
        rx = x.reshape(N*T, -1) # 밑에서 time affine 층으로 올라온 거 (밑: time rnn),
            # 3차원 텐서를 행렬로 바꿔줘야 Affine계산을 생각할 수 있음.

        db = np.sum(dout, axis=0)
        dW = np.dot(rx.T, dout) # Weight matrix에대한 미분. 흘러 들어온 미분 dout에 x를 Transpose해서 엇갈려서 (왼쪽에다가) 곱해줌. (3차원 텐서를 행렬로 바꿔준 rx) 
        dx = np.dot(dout, W.T) # x에 대한 미분. 흘러들어온 미분 dout에 W를 Transpose해서 오른쪽에다가 곱해줌.
        dx = dx.reshape(*x.shape) # 행렬을 다시 원래 형태인 3차원텐서형태로 바꿔줌. Batch_size * Time_size * 그리고 안에는 hidden_size
            # hidden_size인 이유: 왜냐면 x로 미분을 해주는 것인데, x: Time Affine 층으로 들어오는 hidden_state들의 모임이기 때문. (하나하나가 hidden_state shape과 동일해야 함.)

        self.grads[0][...] = dW
        self.grads[1][...] = db
        # 편향벡터, Weight mat 구한 것을 self.grad에 덮어써서 이걸 통해서 update

        return dx # 밑에서 올라온 hidden_state에 대한 미분은 리턴해서 밑(time rnn층)으로 흘려보냄.

# TimeSoftmaxWithLoss class
* ./common/time_layers.py

softmax에 cross entropy를 합친 것. (softmax는 cross entropy와 엮어야 미분공식이 간단해지기 때문.)

In [ ]:
class TimeSoftmaxWithLoss:
    def __init__(self): # initialize할 때 필요한 입력인수는 없음.
        self.params, self.grads = [], []
        ## softmax층 & cross entropy층은 학습해야 할 파라미터가 없음.
        # self.params를 공리스트로 뒀는데 이건 앞으로 채워 넣겠다는게 아니라
        # 그저 다른 층의 클래스와 형식을 통일해주기 위해서 그렇게 한 것.
        # 파라미터를 미분한 그래디언트도 없음.
        # 따라서 self.grads도 채워주려고 만든 게 아니라 그냥 형식 통일시켜주기 위해서 만든 것.
        self.cache = None
        self.ignore_label = -1 # 특정 라벨을 무시하고싶다는 것. label은 0 이상의 정수, -1: 무시하는 라벨 없다는 것.

    def forward(self, xs, ts): # 순전파, xs: 밑(time affine층)에서 올라온 데이터 (어휘들의 score들이 softmax층으로 들어가서 exponential 취해주고 normalize해서 확률분포로 바꿔주는 것...)
        # crossentropy는 소프트맥스가 출력한 확률분포하고 라벨을 원핫인코딩해서 만든 확률분포 둘 사이가 얼마나 가까운지 재는 거니까
        # 라벨 필요. (라벨: ts, 시간에 대한 블록화)
        N, T, V = xs.shape # time affine층에서 나온 스코어들은 3차원 텐서. batch_size * time_size * vocab_size, 안에는 어휘들의 스코어이므로 vocab_size

        if ts.ndim == 3:  # 정답 레이블이 원핫 벡터인 경우. ts: 라벨의 묶음. ts가 3차원 텐서라면...
            # 라벨은 원핫 인코딩이 돼있을 수도 안돼있을 수도.
            # 원핫 인코딩이 안되어있다면 안에 0 이상의 정수가 들어감.
            # 이 경우 ts = 행렬, ndim = 2가 됨.
            ## 원핫인코딩이 돼있다면 0이상의 정수를 원핫벡터로 바꾼거니까 안에 원핫벡터가 들어가 있을 거고, 행렬 안에 벡터가 들어가있는 형태는 3차원 텐서.
            # 따라서 위의 조건식은 행렬안에 원핫벡터가 들어가 있냐는 말. 다시 말해 원핫인코딩이 돼있는지 묻는 말
            ts = ts.argmax(axis=2) # 최대가 일어나는 지점 원핫벡터가 axis = 2, one-hot-vector : 라벨에 해당하는 것만 1, 나머지는 0
                    # 원 핫 벡터에 대해 argmax를 찾는 것은 결국 1인 지점의 인덱스를 찾으란 소리임. (== 라벨찾아라. == one-hot-encoding 돼있는 것을 그 one-hot-encoding 하기 이전으로 바꿔놔라.)

        mask = (ts != self.ignore_label) # self.ignore_label이 -1이라면, 라벨이 -1을 취할리는 없어서 mask는 다 true가 될 것임.
        # 특정라벨 무시하고싶다면 self.ignore_label의 값을 바꾸면 됨.

        # 배치용과 시계열용을 정리(reshape)
        xs = xs.reshape(N * T, V) # 앞(affine 층)에서 했던 것 처럼 밑에서 올라오는 스코어 데이터인 3차원 입력 데이터를 2차원 행렬으로 reshape.
                        # 세로: (Batch_size * Time_size) * 가로: vocab_size
        ts = ts.reshape(N * T) # ts도 마찬가지로 Batch_size * Time_size로 reshape == flatten 해주세요.(안에 comma도 없으니까 벡터도 아니란 소리.)
                               # (reshape하기 전엔 ts 아직 one-hot encoding이 안된 상태로 안에 0 이상의 정수들이 들어있음., 아직 행렬임.)
        mask = mask.reshape(N * T) # mask는 True False로만 구성. reshape을 해주는데 comma가 없으므로 flatten을 해줌.

        ys = softmax(xs) # 행렬로 바꾼 xs 각각에 대해 exponential을 다 취하고, 각 행에 대해 normalize를 해주는 softmax. 그럼 이제 스코어가 확률분포로 바뀜. (최대가 일어나는 지점 동일, 0이상, 합 = 1)
        ls = np.log(ys[np.arange(N * T), ts]) # ys에 대해 슬라이싱 진행., 각각에 대해 로그 취해줌. (이후에 다 더해주고 앞에 - 붙이고 개수로 나눠주고... ==> 이게 정확하게 cross entropy 구하는 과정.)
                        # 좌표는 0 ~ Batch_size * Time_size -1까지, ts는 label들의 묶음을 flatten해서 벡터로 만든 것. (브라켓 안에 있는 두 입력 모두 (N*T)차원 벡터)
                        # ts: 원소들이 라벨로 이루어진 벡터
        ls *= mask  # ignore_label에 해당하는 데이터는 손실을 0으로 설정
        loss = -np.sum(ls)
        loss /= mask.sum() # 평균을 하기 위해 개수로 나누는 것. mask.sum()이니까 True의 개수로 나누는 것. (결국 ignore_label에 해당하는 값은 무시하는 것.)

        self.cache = (ts, ys, mask, (N, T, V))
        return loss

    def backward(self, dout=1): # softmax with loss층이 맨 마지막 층이라서 시작하는 미분이 1임.
        # softmax와 cross entropy를 묶어서 생각하는 이유는 미분공식이 아주 간단하게 y-t로 나오게 되기 때문., y: softmax에서 출력하는 확률 분포, t: label의 원핫벡터
        ts, ys, mask, (N, T, V) = self.cache

        dx = ys
        dx[np.arange(N * T), ts] -= 1 # 슬라이싱 하는 기법. 슬라이싱 한 자리에 1 빼서 dx 업데이트
        # --- softmax with loss 층의 역전파 공식인 y-t임(ts 자리에 1이 들어있어서 그걸 빼는 의미?로 1 빼줌.)
        dx *= dout # 1로 출발해서 곱하나 마나임.
        dx /= mask.sum() # 개수로 나눠줌.
        dx *= mask[:, np.newaxis]  # ignore_labelㅇㅔ 해당하는 데이터는 기울기를 0으로 설정
        # dx에 TF로 이루어진 벡터인 마스크에 축 하나 더 만들어줌. (벡터에 축 하나 더 만들어준다 = 행렬로 만들어준다.)
        # 슬라이싱시 행은 모두. 벡터를 행렬로 이해 하겠다. 근데 이제 열벡터로 이해하겠다.
        # dx에 곱해주라는 의미, True 파트는 1곱하는거니까 살리고, False파트는 0 곱하는 거니까 죽임.
        # 중요한건 아니니까 mask가 헷갈리면 넘어가도 됨.

        dx = dx.reshape((N, T, V)) # dx: Affine층 (밑)으로 흘려보내는 미분.
        # Affine층에서 올라온 score와 shape이 동일해야.
        # 원래 shape인 Batch_size * Time_size * Vocab_size로 reshape

        return dx # score에 대한 미분 리턴. 밑에 있는 affine 층으로 흘려보냄. 밑에있는 Affine층이 이걸 받아서 학습할 것.

# ./ch05/simple_rnnlm.py
RNNLM: RNN Language Model

In [ ]:
# coding: utf-8
import sys
sys.path.append('..')
import numpy as np
from common.time_layers import *

####################################### 여기부터 안들음... 졸았음.
class SimpleRnnlm:
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn

        # 가중치 초기화
        embed_W = (rn(V, D) / 100).astype('f')
        rnn_Wx = (rn(D, H) / np.sqrt(D)).astype('f') # 자비에 초기값 (입력 뉴런개수 표준편차)
        rnn_Wh = (rn(H, H) / np.sqrt(H)).astype('f')
        rnn_b = np.zeros(H).astype('f')
        affine_W = (rn(H, V) / np.sqrt(H)).astype('f')
        affine_b = np.zeros(V).astype('f')

        # 계층 생성
        self.layers = [
            TimeEmbedding(embed_W),
            TimeRNN(rnn_Wx, rnn_Wh, rnn_b, stateful=True),
            TimeAffine(affine_W, affine_b)
        ]
        self.loss_layer = TimeSoftmaxWithLoss()
        self.rnn_layer = self.layers[1]

        # 모든 가중치와 기울기를 리스트에 모은다.
        self.params, self.grads = [], []
        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads

    def forward(self, xs, ts): # 밑에서 단어들 묶음이 들어갈 것(xs, Batch_size*Time_size), ts: 라벨들의 묶음. 내용적으로는 one-hot encoding 안된것
        for layer in self.layers:
            xs = layer.forward(xs) # time embedding 층에서 나온
        loss = self.loss_layer.forward(xs, ts)
        return loss # 전체 ~~ 값 리턴

    def backward(self, dout=1): # dout : y-t
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout) # 역전파
        return dout # 밑으로 흘려보내기

    def reset_state(self):
        self.rnn_layer.reset_state()

# ./ch05/train_custom_loop.py
rnn 언어모델 학습시키기

In [ ]:
# coding: utf-8
import sys
sys.path.append('..')
import matplotlib.pyplot as plt
import numpy as np
from common.optimizer import SGD
from dataset import ptb
from simple_rnnlm import SimpleRnnlm


# 하이퍼파라미터 설정
batch_size = 10 # 말뭉치 10등분, 오른쪽에 있는 걸 밑으로 이어붙이고, 오른쪽에 있는 걸 밑으로 이어붙이고, ... ; 세로: 10
wordvec_size = 100 # 안에있는 건 단어의 id, = corpus 몇 번째 처음 등장하는지.
    # 단어 id가 embedding 층 거치게 되면 단어 id에 해당하는 행을 Weight mat에서 뽑아내게 됨. (그게 이제 단어의 벡터표현)
    # 벡터표현의 차원을 의미
hidden_size = 100 # RNN의 은닉 상태 벡터의 원소 수
    # 벡터표현이 rnn층을지나게 되고. rnn층이 벡터표현을 hidden_state로 바꿔서 출력해줌.
    # (affine 층과 동시에 다음시각의 rnn층에 전달해줘서 과거에 대한 정보를 기억하게 해줌.)
    # hidden_state의 차원을 의미
time_size = 5     # Truncated BPTT가 한 번에 펼치는 시간 크기 # 시간 블럭의 길이를 의미
lr = 0.1
max_epoch = 100

# 학습 데이터 읽기(전체 중 1000개만)
corpus, word_to_id, id_to_word = ptb.load_data('train') # corpus는 단어의 id로 이루어진 list
corpus_size = 1000
corpus = corpus[:corpus_size] # ptb data set에서 앞에 1000개만 생각을 하겠다...
                              # rnn이 성능이 그닥 좋지 못해서 과거에 대한 정보를 그닥 많이 기억하지 못하기 때문
vocab_size = int(max(corpus) + 1) # 단어 id의 최댓값이 결국 1000개(corpus_size)의 말뭉치 안에 있는 어휘 개수,
                                  # +1을 하는 이유는 0부터 시작하기 때문

xs = corpus[:-1]  # 입력                # 맨 오른쪽꺼 제외 # 맨 마지막꺼는 입력해봤자 답이 없으니까.
ts = corpus[1:]   # 출력(정답 레이블)   # 맨 왼쪽꺼 제외
data_size = len(xs) # 맨 끝 제외했으니까 1000-1 (= corpus_size - 1) = 999개
print('말뭉치 크기: %d, 어휘 수: %d' % (corpus_size, vocab_size))

# jump 는 data_size // batch_size = 99

# 학습 시 사용하는 변수
max_iters = data_size // (batch_size * time_size) # 1 epoch동안 학습 몇 번 하냐. (block이 몇 개냐)
                        # 총 데이터를 batch_size * time_size, 즉 블럭 하나 안에 들어있는 데이터의 개수로 나눔
                        # = 블럭의 개수가 나옴.
                        # 몫으로 
                        # 딱 안나눠 떨어지고, 끝에가 조금 남음.
                        # (첫 번째 epoch할 때는 생각을 안하고, 두 번째 epoch부터는 그 조금 남은 부분, 즉 끊긴 부분에서부터 시작해서 결국 데이터를 다 사용하긴 함.)
                # 999 // (10*5) = 19
time_idx = 0 # time_block의 idx
total_loss = 0 # 총 손실함수
loss_count = 0 # 몇 번 했냐 (뒤에서 나눠주기 위해 계속 업데이트 함.)
ppl_list = [] # 퍼플렉시티, 자연어 처리에서 neural.network 성능을 측정하기 위한 지표. (cross entropy와 큰 차이 없음.)

# 모델 생성
model = SimpleRnnlm(vocab_size, wordvec_size, hidden_size) # Rnn 언어 모델을 구현한 클래스,
                                                # 순서대로 time embedding 층, time rnn 층, time affine층, time softmax with loss층 만들고
                                                # 각각으로 instance를 만든 다음에 다 연결한 것
        # 그 rnnlm class로 instance 만든 것.
optimizer = SGD(lr) # stochastic gradient descent

# 미니배치의 각 샘플의 읽기 시작 위치를 계산
jump = (corpus_size - 1) // batch_size # 맨 마지막 단어는 제외해서 corpus_size -1인 것임.
offsets = [i * jump for i in range(batch_size)] # 입력하는 말뭉치를 배치개수만큼 등분해서 맨 앞에 등장하는 단어들의 말뭉치 내에서의 위치.

for epoch in range(max_epoch):
    for iter in range(max_iters):
        # 미니배치 취득
        batch_x = np.empty((batch_size, time_size), dtype='i')
        batch_t = np.empty((batch_size, time_size), dtype='i')
        for t in range(time_size):
            for i, offset in enumerate(offsets):
                batch_x[i, t] = xs[(offset + time_idx) % data_size]
                batch_t[i, t] = ts[(offset + time_idx) % data_size]
            time_idx += 1

        # 기울기를 구하여 매개변수 갱신
        loss = model.forward(batch_x, batch_t)
        model.backward()
        optimizer.update(model.params, model.grads)
        total_loss += loss
        loss_count += 1

    # 에폭마다 퍼플렉서티 평가
    ppl = np.exp(total_loss / loss_count)
    print('| 에폭 %d | 퍼플렉서티 %.2f'
          % (epoch+1, ppl))
    ppl_list.append(float(ppl))
    total_loss, loss_count = 0, 0

# 그래프 그리기
x = np.arange(len(ppl_list))
plt.plot(x, ppl_list, label='train')
plt.xlabel('epochs')
plt.ylabel('perplexity')
plt.show()

# ./ch05/train.py

In [ ]:
# coding: utf-8
import sys
sys.path.append('..')
from common.optimizer import SGD
from common.trainer import RnnlmTrainer
from dataset import ptb
from simple_rnnlm import SimpleRnnlm


# 하이퍼파라미터 설정
batch_size = 10
wordvec_size = 100
hidden_size = 100  # RNN의 은닉 상태 벡터의 원소 수
time_size = 5  # RNN을 펼치는 크기
lr = 0.1
max_epoch = 100

# 학습 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
corpus_size = 1000  # 테스트 데이터셋을 작게 설정
corpus = corpus[:corpus_size]
vocab_size = int(max(corpus) + 1)
xs = corpus[:-1]  # 입력
ts = corpus[1:]  # 출력（정답 레이블）

# 모델 생성
model = SimpleRnnlm(vocab_size, wordvec_size, hidden_size)
optimizer = SGD(lr)
trainer = RnnlmTrainer(model, optimizer)

trainer.fit(xs, ts, max_epoch, batch_size, time_size)
trainer.plot()

# ./trainer.py